# Plot CTD data 

In [1]:
import os
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
from scipy.interpolate import interp1d
from datetime import datetime
import cmocean


%matplotlib widget 

### Parameters

Specify the path to the data folder and the date of the campaign:

In [2]:
data_folder='../data/Profiles/RBR_237207'
date_campaign='20250605'
filenames=["L2_CTD_rbr_237207_20250605_1136_20250605_100637.nc"]

Parameters of the plot

In [16]:
varnames=["Temp","Cond","sat"]
dz_top=0.5 # [m], to remove near the surface
dz_bot=0.5 # [m], to remove at the bottom
ctd_data=[]

### Load the Level-2 data 

In [17]:
for k,file in enumerate(filenames):
    nc=netCDF4.Dataset(os.path.join(data_folder,date_campaign,'Level2',file)) 
    time_num=nc.variables["time"][:].data
    time_date=time_num.astype("datetime64[s]").astype(datetime)
    dict_ctd={"filename":file,"xcoord":getattr(nc, 'X Coordinate (CH1903)') ,"ycoord":getattr(nc, 'Y Coordinate (CH1903)') ,
              "time_start":time_date[0],"depth":nc.variables["depth"][:].data,"time_num":time_num,"time_date":time_date}
    
    for kv in range(len(varnames)): 
        vardata=nc.variables[varnames[kv]][:].data
        if varnames[kv]+"_qual" in nc.variables:
            vardata[nc.variables[varnames[kv]+"_qual"][:]>0]=np.nan
        vardata[dict_ctd["depth"]<dz_top]=np.nan
        vardata[dict_ctd["depth"]>(np.nanmax(dict_ctd["depth"])-dz_bot)]=np.nan
    ctd_data.append(dict_ctd)
    nc.close()